## Imports

In [1]:
from datasets import load_dataset, ClassLabel

In [2]:
VERSION = "v2"
!ls -lah datasets | grep {VERSION} | grep piqa
# !ls -lah datasets | grep {VERSION} | grep qnli
# !ls -lah datasets | grep {VERSION} | grep imdb
# !ls -lah datasets | grep {VERSION}

drwxr-xr-x  3 augustas Domain Users  25K Jul 24 20:31 wrapped_piqa_ppo_training_raw_v2
-rw-r--r--  1 augustas Domain Users 972K Jul 24 21:26 wrapped_piqa_ppo_training_v2.parquet
drwxr-xr-x  3 augustas Domain Users  33K Jul 24 20:31 wrapped_piqa_train_raw_v2
-rw-r--r--  1 augustas Domain Users 1.4M Jul 24 21:36 wrapped_piqa_train_v2.parquet
-rw-r--r--  1 augustas Domain Users 246K Jul 24 21:41 wrapped_piqa_validation_v2.parquet


In [9]:
# parquet_data_files = {
#     "train": f"datasets/burns_datasets_VINC_train_{VERSION}.parquet",
#     "validation": f"datasets/burns_datasets_VINC_validation_{VERSION}.parquet",
# }
# parquet_data_files = {
#     "train": f"datasets/burns_datasets_VINC_imdb_train_{VERSION}.parquet",
#     "validation": f"datasets/burns_datasets_VINC_imdb_validation_{VERSION}.parquet",
# }
# parquet_data_files = {
#     "train": f"datasets/qnli_vicuna_train_{VERSION}.parquet",
#     "validation": f"datasets/qnli_vicuna_validation_{VERSION}.parquet",
# }
# parquet_data_files = {
#     "train": f"datasets/wrapped_piqa_train_{VERSION}.parquet",
#     "validation": f"datasets/wrapped_piqa_validation_{VERSION}.parquet",
# }

# parquet_data_files = {
#     "train": f"datasets/burns_datasets_VINC_ppo_training_{VERSION}.parquet",
# }
# parquet_data_files = {
#     "train": f"datasets/burns_datasets_VINC_imdb_ppo_training_{VERSION}.parquet",
# }
# parquet_data_files = {
#     "train": f"datasets/qnli_vicuna_ppo_training_{VERSION}.parquet",
# }
parquet_data_files = {
    "train": f"datasets/wrapped_piqa_ppo_training_{VERSION}.parquet",
}

my_dataset = load_dataset("parquet", data_files=parquet_data_files)
my_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /admin/home-augustas/.cache/huggingface/datasets/parquet/default-9dc0c8adbe89b2fc/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['original_dataset', 'template_name', 'prompt', 'best_response'],
        num_rows: 6113
    })
})

In [10]:
if len(my_dataset.keys()) == 2:
    for split in my_dataset:
        print(split)
        
        features = my_dataset[split].features
        names = ["neg", "pos"]
        if "qnli" in parquet_data_files["train"]: names = ["entailment", "not_entailment"]
        
        features["label"] = ClassLabel(names=names)
        my_dataset[split] = my_dataset[split].map(features=features, load_from_cache_file=False)
        print(my_dataset[split].features["label"])
else:
    print("PPO dataset - only one split")

PPO dataset - only one split


In [11]:
# if len(my_dataset.keys()) == 2:
#     print("Train/validation dataset")
#     # my_dataset.push_to_hub(f"AugustasM/burns-datasets-VINC-{VERSION}", private=True)
#     # my_dataset.push_to_hub(f"AugustasM/burns-datasets-VINC-imdb-{VERSION}", private=True)
#     # my_dataset.push_to_hub(f"AugustasM/qnli-vicuna-{VERSION}", private=True)
#     my_dataset.push_to_hub(f"AugustasM/wrapped-piqa-{VERSION}", private=True)
# else:
#     print("PPO dataset")
#     # my_dataset.push_to_hub(f"AugustasM/burns-datasets-VINC-ppo-training-{VERSION}", private=True)
#     # my_dataset.push_to_hub(f"AugustasM/burns-datasets-VINC-imdb-ppo-training-{VERSION}", private=True)
#     # my_dataset.push_to_hub(f"AugustasM/qnli-vicuna-ppo-training-{VERSION}", private=True)
#     my_dataset.push_to_hub(f"AugustasM/wrapped-piqa-ppo-training-{VERSION}", private=True)

Pushing split train to the Hub.


PPO dataset


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

## Test

In [12]:
# tmp = load_dataset(f"AugustasM/burns-datasets-VINC-{VERSION}")
# tmp = load_dataset(f"AugustasM/burns-datasets-VINC-imdb-{VERSION}")
# tmp = load_dataset(f"AugustasM/qnli-vicuna-{VERSION}")
tmp = load_dataset(f"AugustasM/wrapped-piqa-{VERSION}")

tmp["train"].features["label"], tmp["validation"].features["label"]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /admin/home-augustas/.cache/huggingface/datasets/AugustasM___parquet/AugustasM--wrapped-piqa-v2-66c8bf361528121d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

(ClassLabel(names=['neg', 'pos'], id=None),
 ClassLabel(names=['neg', 'pos'], id=None))

In [13]:
tmp

DatasetDict({
    train: Dataset({
        features: ['label', 'original_dataset', 'template_name', 'text'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['label', 'original_dataset', 'template_name', 'text'],
        num_rows: 1838
    })
})

In [14]:
# import datasets
# datasets.disable_progress_bar()
# datasets.logging.set_verbosity_error()


# for i in range(1, 6):
#     tmp = load_dataset(f"AugustasM/burns-datasets-VINC-imdb-v1{i}")
#     print(tmp["train"][0]["text"][:100])
#     print(tmp["validation"][0]["text"][:100])
#     print("-" * 80)

In [15]:
# tmp = load_dataset(f"AugustasM/burns-datasets-VINC-ppo-training-{VERSION}")
# tmp = load_dataset(f"AugustasM/burns-datasets-VINC-imdb-ppo-training-{VERSION}")
# tmp = load_dataset(f"AugustasM/qnli-vicuna-ppo-training-{VERSION}")
tmp = load_dataset(f"AugustasM/wrapped-piqa-ppo-training-{VERSION}")
tmp

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6113 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /admin/home-augustas/.cache/huggingface/datasets/AugustasM___parquet/AugustasM--wrapped-piqa-ppo-training-v2-0db68068d06638e6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['original_dataset', 'template_name', 'prompt', 'best_response'],
        num_rows: 6113
    })
})